In [124]:
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 
import numpy as np
import pandas as pd
import csv

In [4]:
import gensim
from nltk.data import find
nltk.download("word2vec_sample")
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

[nltk_data] Downloading package word2vec_sample to
[nltk_data]     /home/ilana/nltk_data...
[nltk_data]   Package word2vec_sample is already up-to-date!


In [ ]:
happy_vector = word_vectors[word_list.index("happy")].toarray()[0]
legislation_vector = word_vectors[word_list.index("legislation")].toarray()[0]


def get_top_10_closest_words(word,word_vectors,word_list):
    word_index = word_list.index(word)
    word_vector = word_vectors[word_index]
    if word_vector.shape[0] != 1:
        word_vector = [word_vector]
    comparisons = cosine_similarity(word_vector,word_vectors)
    for index in reversed(np.argsort(comparisons[0])[-10:]):
        print(word_list[index])

In [ ]:
def compare_similarity(similarity_dict1,similarity_dict2):
    X = [similarity_dict1[word_pair] for word_pair in similarity_dict2]
    Y = [similarity_dict2[word_pair] for word_pair in similarity_dict1]
    return pearsonr(X,Y)[0]

In [19]:
def preprocess(text):
    '''tokenize text into list of lists, sents and words, remove stopwords'''
    stop_words = set(stopwords.words('english')) 
    sents = sent_tokenize(text)
    result = []
    for sent in sents:
        word_tokenized = word_tokenize(sent)
        result.append([w for w in word_tokenized if not w in stop_words] )
    return result

In [14]:
#change in topic
def get_cosine_similarities(sents, word_vectors, k): # k = len(sents)
    
    ''' returns an array of similarities of len(sents), corresponding to the cosine of the
    vectors produced by summing the embeddings of words in the k sentences of either side of 
    each sentence boundary, with the first and last k similarities set to -1
    ---------------------
    sents - list, word tokenized'''
    
    cosine_similarities = [-1]*k
    #your code here
    sents_vec = [] #one embedding per sentence
    for sent in sents:
        sent_vec = []
        for word in sent:
            if word in word2vec_model:
                word_vec = word2vec_model.wv[word]
                sent_vec.append(word_vec) # add each word embedding per sent
        sents_vec.append(np.sum(sent_vec, axis = 0)) # sum over columns
   
    for i in range(len(sents_vec) - k):
        if 2*k+i <= len(sents_vec)-1:
            cluster_1 = np.sum(sents_vec[i:k+i], axis = 0)
            cluster_2 = np.sum(sents_vec[(k + i):(k + i)+k], axis = 0)
            cosine_similarities.append(1 - cosine(cluster_1, cluster_2))

    for i in range(k):
        cosine_similarities.append(-1)
    
    return np.array(cosine_similarities)

# cosine_similarities = get_cosine_similarities(sents,word2vec_model,3)


In [105]:
#change in topic
def get_text_embedding(sents, word_vectors): # k = len(sents)
    
    ''' returns an word embeddings of len(sents), produced by summing the embeddings of words 
    in each sentences
    ---------------------
    sents - list, word tokenized
    word_vectors - word2vec model'''
    
    #your code here
    sents_vec = [] #one embedding per sentence
    for sent in sents:
        sent_vec = []
        for word in sent:
            if word in word2vec_model:
                word_vec = word2vec_model.wv[word]
                sent_vec.append(word_vec) # add each word embedding per sent
        sents_vec.append(np.sum(sent_vec, axis = 0)) # sum over columns
   
    result = np.sum(sents_vec, axis = 0)
    
    return result.flatten()

# cosine_similarities = get_cosine_similarities(sents,word2vec_model,3)


In [97]:
test = preprocess("Hi, this is a test. Whats up. Im tired as heck.")
get_text_embedding(test, word2vec_model).shape

/home/ilana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  app.launch_new_instance()


(300,)

In [62]:
def get_tokenized_docs(all_inputs_archive):
    ''' Given all documented user cases dict, return dict of doc_id to word-tokenized sents
    ------------
    all_inputs_archive: dictionary where key = user_id, value = raw text'''
    result = dict()
    for doc, text in all_inputs_archive.items():
        result[doc] = preprocess(text)
    return result

In [127]:
# process text and generate single vector for text

def get_similarities(input_text_vec, archive_preprocessed_dict):
    ''' Takes in embedding of input text (dense matrix) and preprocessed dictionary of user_id: tokenized_text.
    Returns dictionary of cosine similaries where key is user_id, output is cosine similarity between archive text and input text'''
    text_similarities = dict()
    for usr_id, text in archive_processed.items():
        text_vec = get_text_embedding(text, word2vec_model)
        text_similarities[usr_id] = 1 - cosine(input_vec, text_vec)
    return text_similarities

def output_most_similar_text(similarities_dict, archive_df):
    '''Given dictionary mapping similarities to cosine_similarity
    and data frame of full archive text, return csv file with original fields and similarity for most similar text.
    -------------------
    similarities_dict: dict, user_id:cos_similarity
    archive_df: pandas DataFrame of archive data'''
    max_sim = 0
    best_usr = ""
    for key, val in similarities_dict.items():
        if val > max_sim:
            max_sim = val
            best_usr = key
#     output[best_usr] =  max_sim
    result = archive_df[archive_df["Unique ID"] == best_usr]
    result["similarity"] = max_sim
    
    return result.to_csv(r'../Code/results.csv')



In [134]:
archive_df = pd.read_csv("../Data/archive.csv")
archive_df.dropna(inplace = True)
keys = archive_df['Unique ID'].iloc[1:]
vals = archive_df['Details'].iloc[1:]
archive_dict  =  dict(zip(keys, vals))

In [140]:
archive_df[archive_df["Unique ID"] == 0].loc[:,"Details"][0]

'After disagreeing with my manager, I was taken aside and given a severance offer, while my male coworker who agreed with me was kept on the team'

In [129]:
input_text = archive_df['Details'][0]                            
sents_input = preprocess(input_text)
input_vec = get_text_embedding(sents_input, word2vec_model)
archive_processed = get_tokenized_docs(archive_dict)

/home/ilana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  app.launch_new_instance()


In [130]:
input_vec.flatten().shape

(300,)

In [132]:
similarities_dict = get_similarities(input_vec, archive_processed)
output_most_similar_text(similarities_dict, archive_df)

/home/ilana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  app.launch_new_instance()
/home/ilana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [110]:
archive_dict

{1: 'In team meetings where my coworkers and I share anecdotes about our personal lives, I mentioned my preference for they/them pronouns. My manager was visibly uncomfortable, and a few weeks later I was transferred to a different team, to a role that is less senior.',
 2: 'After 5 years working at my bank, consistently earning positive performance reviews and being promised a raise by my boss, my newer Canadian co-worker was promoted before me. My boss said it was because a Canadian would be better suited for this position than me.',
 3: 'I was unable to claim my insurance reimbursement when the human resources representative learned that I was First Nations, claiming that I already get too many handouts because of my First Nations status.',
 4: 'The head cook and male kitchen staff have consistently made sexually charged comments about how I look in my work uniform. I have told them that this is unacceptable behavior, but they have continued to do this, with no intervention from my 

In [133]:
similarities_dict

{1: 0.6889528632164001,
 2: 0.6368640065193176,
 3: 0.569545567035675,
 4: 0.6365067958831787,
 10: 0.7169502377510071}

In [119]:
archive_df["Details"].iloc[0]

'After disagreeing with my manager, I was taken aside and given a severance offer, while my male coworker who agreed with me was kept on the team'

In [118]:
archive_df['Details'].iloc[5]

'Given more responsibility in my role, but not more pay. I learned that my male coworker who was just hired was getting paid more for the same responsibilities, and when I brought this up to my manager, I was fired and banned from the premises.'

In [123]:
archive_df[archive_df["Unique ID"] == 10]

,Unique ID,What happened?,Who is the employer?,Why do you believe this happened?,What Provice?,Details,Resolution
10,10,not_promoted,Airport or Airline,genetic,Saskatchewan,"Given more responsibility in my role, but not ...",Learned about my rights and consulted with a l...
